In [30]:
!ls
# Numerical Imports 
import pandas as pd
import numpy as np
import scipy 

# Plotting 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# %matplotlib notebook

# Python 
import os

# sklearn 
from sklearn.metrics import f1_score # f1_score(y_true, y_pred)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel

# scipy
from scipy.cluster import hierarchy as hc # for dendograms 

Full.ipynb	     NA with RNA-Seq.ipynb  sum_tab_1.csv  train_cli.tsv
Gender Visual.ipynb  Proteomics.ipynb	    sum_tab_2.csv  train_pro.tsv
Import.ipynb	     README.txt		    test_cli.tsv   train_rna.tsv
MSI Visual.ipynb     RNA-Seq.ipynb	    test_pro.tsv   visuals.ipynb
NA.ipynb	     start.ipynb	    test_rna.tsv


In [31]:
df_train_pro = pd.read_table(f'{os.getcwd()}/train_pro.tsv', 
                           delim_whitespace=True,
                           low_memory=False,).T
df_test_pro = pd.read_table(f'{os.getcwd()}/test_pro.tsv', 
                           delim_whitespace=True,
                           low_memory=False,).T
df_train_rna = pd.read_table(f'{os.getcwd()}/train_rna.tsv', 
                           delim_whitespace=True,
                           low_memory=False,).T
df_test_rna = pd.read_table(f'{os.getcwd()}/test_rna.tsv', 
                           delim_whitespace=True,
                           low_memory=False,).T
df_train_cli = pd.read_csv(f'{os.getcwd()}/train_cli.tsv', 
                           delim_whitespace=True,
                           low_memory=False,)
df_test_cli = pd.read_csv(f'{os.getcwd()}/test_cli.tsv', 
                           delim_whitespace=True,
                           low_memory=False,)
df_train_mislabel = pd.read_csv(f'{os.getcwd()}/sum_tab_1.csv', 
                           low_memory=False,)
pd.set_option("display.max_rows", 250)
pd.set_option("display.max_columns", 250)

In [32]:
# df_train_pro
# df_test_pro
# df_train_rna
# df_test_rna
# df_train_cli
# df_test_cli
# df_train_mislabel

In [33]:
# Come back to the way you handle this NA, sophisticated way will imporve by at least 5% 
train_pro = df_train_pro.copy(deep=True)
# train_pro = train_pro.fillna(train_pro.median())
train_pro.index.name = 'sample'

test_pro = df_test_pro.copy(deep=True)
# test_pro = test_pro.fillna(test_pro.median())
test_pro.index.name = 'sample'


train_rna = df_train_rna.copy(deep=True)
# train_rna = train_rna.fillna(train_rna.median())
train_rna.index.name = 'sample'

test_rna = df_test_rna.copy(deep=True)
# test_rna = test_rna.fillna(test_rna.median())
test_rna.index.name = 'sample'

In [34]:
# df_train_pro
# train_pro
# df_test_pro
# test_pro

In [35]:
# df_train_rna
# train_rna
# df_test_rna
# test_rna

In [36]:
train_cli = df_train_cli.copy(deep=True)
train_cli = train_cli.set_index('sample')
train_cli = train_cli.replace({'gender': {'Male':0, 'Female':1},
                              'msi': {'MSI-Low/MSS':0, 'MSI-High':1}})

test_cli = df_test_cli.copy(deep=True)
test_cli = test_cli.set_index('sample')
test_cli = test_cli.replace({'gender': {'Male':0, 'Female':1},
                              'msi': {'MSI-Low/MSS':0, 'MSI-High':1}})

In [37]:
# df_train_cli
# train_cli
# df_test_cli
# test_cli

In [38]:
train_mislabel = df_train_mislabel.copy(deep=True)
train_mislabel = train_mislabel.set_index('sample')

In [39]:
# df_train_mislabel
# train_mislabel

In [40]:
train_pro.reset_index(drop=True, inplace=True)
train_rna.reset_index(drop=True, inplace=True)
train_cli.reset_index(drop=True, inplace=True)
train_mislabel.reset_index(drop=True, inplace=True)

In [41]:
# train_pro
# train_rna
# train_cli
# train_mislabel

In [42]:
train_pro_combined = pd.concat([train_mislabel, train_cli, train_pro], axis=1)
train_rna_combined = pd.concat([train_mislabel, train_cli, train_rna], axis=1)
train_combined = pd.concat([train_mislabel, train_cli, train_rna, train_pro], axis=1)

In [43]:
# train_pro_combined
# train_rna_combined
train_combined

,mismatch,gender,msi,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A4GALT,AAAS,AACS,AADAC,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA10,ABCA11P,ABCA12,ABCA13,ABCA17P,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,ABCA8,ABCA9,ABCB1,ABCB10,ABCB11,ABCB4,ABCB5,ABCB6,ABCB7,ABCB8,ABCB9,ABCC1,ABCC10,ABCC11,ABCC13,ABCC2,ABCC3,ABCC4,ABCC5,ABCC6,ABCC6P1,ABCC6P2,ABCC8,ABCC9,ABCD1,ABCD2,ABCD3,ABCD4,ABCE1,ABCF1,ABCF2,ABCF3,ABCG1,ABCG2,ABCG4,ABCG5,ABCG8,ABHD1,ABHD10,ABHD11,ABHD12,ABHD12B,ABHD13,ABHD14A,ABHD14B,ABHD15,ABHD16A,ABHD16B,ABHD2,ABHD3,ABHD4,ABHD5,ABHD6,ABHD8,ABI1,ABI2,ABI3,ABI3BP,ABL1,ABL2,ABLIM1,ABLIM2,ABLIM3,ABO,ABR,ABRACL,ABT1,ABTB1,ABTB2,ACAA1,ACAA2,ACACA,ACACB,ACAD10,ACAD11,ACAD8,ACAD9,ACADM,ACADS,ACADSB,ACADVL,ACAN,ACAP1,...,VPS18,VPS25,VPS26A,VPS26B,VPS28,VPS29,VPS33A,VPS33B,VPS35,VPS36,VPS37B,VPS41,VPS45,VPS4A,VPS4B,VPS50,VPS51,VPS52,VPS53,VRK1,VSIG4,VTA1,VTI1B,VTN,VWA1,VWA5A,VWA8,VWF,WAPL,WARS,WARS2,WAS,WASF2,WASHC1,WASHC3,WASHC4,WASHC5,WASL,WBP11,WBP2,WDFY1,WDR1,WDR11,WDR12,WDR13,WDR18,WDR26,WDR3,WDR36,WDR4,WDR43,WDR44,WDR45,WDR5,WDR61,WDR74,WDR75,WDR77,WDR82,WFS1,WRNIP1,WTAP,WWC3,XAB2,XIAP,XPC,XPNPEP1,XPNPEP2,XPNPEP3,XPO1,XPO4,XPO5,XPO7,XPOT,XRCC1,XRCC4,XRCC5,XRCC6,XRN1,XRN2,YARS,YARS2,YBX1,YBX3,YIPF4,YIPF5,YIPF6,YKT6,YLPM1,YME1L1,YTHDF2,YTHDF3,YWHAB,YWHAE,YWHAG,YWHAH,YWHAQ,YWHAZ,ZADH2,ZAK,ZBED1,ZBTB7A,ZC3H11A,ZC3H13,ZC3H14,ZC3H15,ZC3H4,ZC3HAV1,ZC3HAV1L,ZCCHC8,ZFPL1,ZFR,ZG16,ZMPSTE24,ZMYM3,ZNF185,ZNF207,ZNF280C,ZNF326,ZNF638,ZNF706,ZPR1,ZW10,ZYX,ZZEF1
0,0,1,0,0.810976,0.294094,2.898144,6.548415,1.004192,2.519397,5.749835,5.043260,0.297011,4.504254,2.024335,5.403343,1.831152,5.165514,7.240806,NaN,5.403007,0.409724,6.551073,3.100041,5.277979,3.298451,5.045930,1.001300,5.689962,0.969101,2.982777,2.366695,0.245197,3.066077,1.492358,0.053929,0.031171,3.255512,0.519140,0.096053,2.246003,0.349446,1.483484,1.022936,0.511371,3.912004,3.714960,0.155912,0.653988,0.175236,3.802488,4.574414,3.433086,1.912922,3.343587,3.305114,0.103883,1.015897,4.427634,4.611711,4.095616,3.138293,3.543970,1.999173,2.875465,0.046088,3.028601,2.024893,0.196113,5.713165,4.111305,6.101022,4.169641,5.450756,4.993056,3.230541,1.144649,0.160974,0.173802,0.144580,0.341795,4.883685,6.468786,6.234869,NaN,3.294638,4.411262,5.705954,3.380753,4.866446,1.859904,3.859852,5.033883,4.344119,3.032352,3.096348,1.994397,5.443316,2.963861,2.109574,0.996480,4.188414,2.307384,4.654947,2.092253,1.161089,3.701283,5.096069,7.270606,5.695157,4.313367,2.516734,7.325857,8.451549,4.358400,1.960287,3.472469,2.684599,4.044159,4.867411,5.169815,5.370055,3.156983,7.553656,0.447867,1.687745,...,1.874889,2.102446,3.644706,0.999049,2.352649,1.355122,1.700971,0.986234,5.201797,1.694419,2.016128,1.037863,2.129760,NaN,2.439649,1.121213,2.570043,1.978001,2.120013,1.037239,NaN,1.985583,1.402441,3.587076,3.139336,1.604568,2.042865,3.276686,1.032778,5.450480,1.551416,1.061254,2.249946,NaN,1.032550,2.360181,3.720805,1.909891,NaN,0.984047,2.031655,6.068300,2.420489,2.353787,1.001043,1.004630,1.065994,2.221940,3.515957,1.023543,1.869788,NaN,NaN,2.177536,3.296747,1.004654,1.794322,1.570204,2.553235,1.925916,NaN,0.991065,1.295359,1.031230,NaN,NaN,3.648148,NaN,2.108976,4.665956,1.133668,3.241074,3.263931,3.196612,2.676144,0.969840,5.853167,5.692673,NaN,3.632066,4.390283,2.785913,2.733676,2.341250,NaN,1.006993,NaN,2.698599,1.473819,1.917234,1.455458,1.493388,4.193716,5.192265,3.821649,4.017592,4.665611,6.300753,1.915149,1.620717,NaN,1.030688,0.992511,1.652999,NaN,1.630672,1.051260,2.915539,1.624048,1.040985,2.097838,2.657478,5.774241,2.280993,NaN,NaN,2.078553,NaN,1.505944,1.007971,1.005565,1.016313,1.859706,3.834986,NaN
1,1,1,0,0.340575,0.286721,2.223730,7.355751,0.967359,2.042327,5.949647,5.540432,0.633366,3.143702,2.852460,5.214069,1.350142,4.827544,7.442804,0.077730,4.881468,0.157388,6.859313,3.773362,4.681637,2.354399,3.245056,2.188932,6.038234,0.801714,1.874068,2.420736,0.830577,2.926069,0.687222,0.172355,NaN,3.612654,1.402553,0.166862,2.521407,0.83262

In [44]:
train_combined[[ # msi proteomics
                'TAP1', 'APOL2', 'LCP1', 'PTPN6', 'CASK', 'UBE2L6', 'ICAM1', 'ITGB2',
                'SDF2L1', 'CKB', 'LAP3', 'IFI30', 'PTPRC', 'HSDL2', 'RFC2', 'WARS',
                'IFI35', 'TYMP', 'CSRP2', 'TAPBP', 'ERMP1', 'ANP32E', 'HP', 'HK3',
                'ROCK2', 'ADPGK', 'HSPA4L', 'CNDP2', 'RFTN1', 'GBP1', 'GBP2', 'FMNL1',
                'NCF2', 'YARS2', 'RPL3', 'SPTLC2', 'ENO1', 'SNX12', 'DMBT1', 'ARL3',
                
                 # mis rna-seq 
                'EPDR1', 'APOL3', 'POU5F1B', 'CFTR', 'CIITA', 'RAB32', 'MAX',
                'PRSS23', 'FABP6', 'GABRP', 'LAP3', 'LY6G6D', 'SLC19A3', 'WARS',
                'DLGAP1-AS5', 'GBP1', 'RAMP1', 'AREG', 'GSPT2', 'TNFAIP2',
                'EREG', 'TNNC2', 'ANKRD27', 'PLCL2', 'TFCP2L1', 'LAG3', 'GRM8', 
                'BEX2', 'DEFB1', 'FABP1', 'IRF1', 'CCL4', 'SLC25A48', 'SLC51B', 
                'TRIM72', 'GBP4', 'HPSE',
                
                # gender proteomics
                'MYO9B', 'EMILIN1', 'MRPL16', 'EIF4G2', 'TNXB', 'ARFIP1', 'GAR1', 'SLC35A2',
                'GADD45GIP1', 'TJP2', 'ALDH1A1', 'PGM2', 'LTBP1', 'NNMT', 'COX7C', 'FBN1',
                'SERPINB1', 'U2AF1L5', 'COG3', 'ASPN', 'METTL1', 'DDT', 'FBLN1', 'FLNC', 
                'NAP1L1', 'AGRN', 'RCC2', 'ZNF706', 'C4A', 'COL4A2', 'S100A14', 
                    
                # gender rna-seq 
                'RPS4Y1', 'ZFX', 'DDX3Y', 'UTY', 'SRSF6', 'GYG2P1', 'ZRSR2', 
                'EVPL', 'LUC7L2', 'SRGAP2B', 'NAT8', 'ZNF862', 'ODAM', 'RGS4', 
                'RNASEH2A', 'EIF1AY', 'FOXD2', 'PMS2P3', 'MND1', 'FAM3D', 'DDB2', 
                'DDX3X', 'NINJ2', 'NLRX1', 'COCH', 'IRF5', 'FHDC1', 'ZNF606', 
                'FBXW9', 'RPL21P28', 'TLX1', 'HEBP2', 'TRIM29', 'ZNF273', 'KCNJ2', 
                'CRABP2', 'ABCA6', 'MMP17', 'PCNA', 'S100A16', 'UBA6', 'EMX1', 'EHBP1', 'THG1L',
                ]].isnull().sum(axis = 0)

TAP1           0
TAP1           0
APOL2          0
APOL2         15
LCP1           0
LCP1           0
PTPN6          0
PTPN6          0
CASK           0
CASK           2
UBE2L6         0
UBE2L6        15
ICAM1          0
ICAM1          1
ITGB2          0
ITGB2          1
SDF2L1         0
SDF2L1        24
CKB            0
CKB            3
LAP3           0
LAP3           0
IFI30          0
IFI30         36
PTPRC          0
PTPRC          4
HSDL2          0
HSDL2          0
RFC2           0
RFC2           9
WARS           0
WARS           0
IFI35          0
IFI35          5
TYMP           0
TYMP           0
CSRP2          0
CSRP2         16
TAPBP          0
TAPBP          2
ERMP1          0
ERMP1          1
ANP32E         0
ANP32E         0
HP            10
HP             0
HK3            0
HK3           10
ROCK2          0
ROCK2          2
ADPGK          0
ADPGK         14
HSPA4L         1
HSPA4L        37
CNDP2          0
CNDP2          0
RFTN1          0
RFTN1          7
GBP1          

In [46]:
train_combined[[ # msi proteomics
                'TAP1', 'LCP1', 'PTPN6', 'CASK', 'ICAM1', 'ITGB2',
                'CKB', 'LAP3', 'PTPRC', 'HSDL2', 'WARS',
                'IFI35', 'TYMP', 'TAPBP', 'ERMP1', 'ANP32E',
                'ROCK2', 'CNDP2', 'RFTN1', 'GBP1',
                'NCF2', 'YARS2', 'RPL3', 'ENO1', 'SNX12', 'ARL3',
                
                 # mis rna-seq 
                'EPDR1', 'APOL3', 'POU5F1B', 'CFTR', 'CIITA', 'MAX',
                'PRSS23', 'FABP6', 'GABRP', 'LAP3', 'SLC19A3', 'WARS',
                'GBP1', 'RAMP1', 'AREG',
                'EREG', 'TNNC2', 'ANKRD27', 'PLCL2', 'TFCP2L1', 'LAG3', 'GRM8', 
                'BEX2', 'DEFB1', 'IRF1', 'CCL4', 'SLC51B', 
                'GBP4', 'HPSE',
                
                # gender proteomics
                'EMILIN1', 'EIF4G2', 'ARFIP1', 'GAR1',
                'TJP2', 'ALDH1A1', 'PGM2', 'LTBP1', 'NNMT', 'COX7C', 'FBN1',
                'SERPINB1', 'U2AF1L5', 'COG3', 'FBLN1', 'FLNC', 
                'NAP1L1', 'AGRN', 'RCC2', 'COL4A2', 'S100A14', 
                    
                # gender rna-seq 
                'ZFX', 'SRSF6', 'ZRSR2', 
                'EVPL', 'LUC7L2', 'ZNF862', 'ODAM', 'RGS4', 
                'FOXD2', 'PMS2P3', 'MND1', 'FAM3D', 'DDB2', 
                'DDX3X', 'NINJ2', 'COCH', 'IRF5', 'FHDC1', 'ZNF606', 
                'FBXW9', 'RPL21P28', 'TLX1', 'HEBP2', 'TRIM29', 'ZNF273', 'KCNJ2', 
                'CRABP2', 'ABCA6', 'MMP17', 'PCNA', 'S100A16', 'UBA6', 'EHBP1', 'THG1L',
                ]].isnull().sum(axis = 0)

TAP1        0
TAP1        0
LCP1        0
LCP1        0
PTPN6       0
PTPN6       0
CASK        0
CASK        2
ICAM1       0
ICAM1       1
ITGB2       0
ITGB2       1
CKB         0
CKB         3
LAP3        0
LAP3        0
PTPRC       0
PTPRC       4
HSDL2       0
HSDL2       0
WARS        0
WARS        0
IFI35       0
IFI35       5
TYMP        0
TYMP        0
TAPBP       0
TAPBP       2
ERMP1       0
ERMP1       1
ANP32E      0
ANP32E      0
ROCK2       0
ROCK2       2
CNDP2       0
CNDP2       0
RFTN1       0
RFTN1       7
GBP1        0
GBP1        3
NCF2        0
NCF2        4
YARS2       0
YARS2       0
RPL3        0
RPL3        0
ENO1        0
ENO1        0
SNX12       0
SNX12       3
ARL3        0
ARL3        6
EPDR1       0
APOL3       0
POU5F1B     1
CFTR        0
CIITA       0
MAX         0
PRSS23      0
FABP6       0
GABRP       2
LAP3        0
LAP3        0
SLC19A3     0
WARS        0
WARS        0
GBP1        0
GBP1        3
RAMP1       0
AREG        0
EREG        0
TNNC2 